In [1]:
# import matplotlib
# matplotlib.use("qtagg")
%matplotlib WebAgg
import os
root = os.path.realpath(os.getcwd()+ os.sep+"..")
import sys
sys.path.append(root)
from modules.WiFi import WiFi, Modes, make_command
import pathfinder, pathfinder_visualizer
import importlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import time
import re, json
from datetime import datetime
import threading

In [2]:
def show_visual():
    def on_hover_point(point_polar):
        pathfinder.set_goal(point_polar) if not point_polar is None else None
    fig, update_func = pathfinder_visualizer.run_visualizer(pathfinder, on_hover_point)
    anime = anim.FuncAnimation(fig, update_func, 1, interval=50, blit=True)
    plt.show()
    return anime
# show_visual()

from ipywidgets import Button, VBox, ToggleButton, IntSlider, HBox, HTML, Layout, Output

start_pathfinder = Button(
    description="Start Pathfinder",
    layout = Layout(width='auto', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="primary",
    disabled = pathfinder._service.is_running())
stop_pathfinder = Button(
    description="Stop Pathfinder", 
    layout= start_pathfinder.layout, 
    style=start_pathfinder.style,
    button_style="warning",
    disabled = not pathfinder._service.is_running())
run_rover = ToggleButton(
    value=False, 
    description="Run Rover", 
    layout= Layout(flex='1', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="success",
    disabled = not pathfinder._service.is_running())
visual_button = Button(
    description="PLT", 
    layout = Layout(height='auto', margin='10px'))
speed_slide = IntSlider(
    value=0, min=0, max=100, 
    orientation="vertical", 
    layout=Layout(height="auto"))
# text_display = HTML(
#     value="Some HTML text", 
#     # layout= Layout(flex='1', margin="10px", border="solid", padding="10px"))
#     layout= )
text_output = Output(layout=Layout(flex='1', max_height="500px", min_height="100px"))
poll_rover = Button(
    description="Poll Rover",
    layout = Layout(flex='1')
)
clear_output = Button(
    description="Clear Output"
)

buttons = VBox([
    start_pathfinder,
    HBox([run_rover,], layout=Layout(width='auto', padding='0px', margin='0px')),
    stop_pathfinder,
    ],
    layout = Layout(flex='1')
)
controls = HBox([
    speed_slide, buttons, speed_slide
])
logging = HBox([
    HBox([text_output], layout=Layout(border='solid', flex='1', padding='10px')), VBox([poll_rover, clear_output])
    ],
    layout = Layout(margin='10px')
)
gui = VBox([
    controls,
    logging
])
# buttons

called start_path_finder_service
Starting Pathfinder Service
Called start_worldview_service
Starting Worldview Service
Called start_lidar_service
Starting Lidar Service

started Lidar Scanning Thread
called stop_path_finder_service

Called stop_worldview_service
Called stop_lidar_service
waiting for obstacle thread to join
waiting for lidar thread to join
Stopping Lidar Service
Starting Worldview Service
Stopping Pathfinder Service


In [3]:
from math import pi
from unified_utils import Service

rover = WiFi("http://192.168.0.211:5000")
def captain_act():
    path = pathfinder.get_path()
    if len(path) < 2:
        rover.send_command(make_command(speed_percent=0))
        # speed_slide.value=0
        radians=pi/2
    else:
        nxt = path[1]
        radians = (nxt[0]%(2*pi))-pi/2
    radians*=-1
    degrees = int(radians/(2*pi)*360)
    mode = Modes.DRIVE if abs(degrees) < 12 else Modes.SPIN
    speed = speed_slide.value
    if mode == Modes.SPIN:
        speed *= (-1 if degrees < 0 else 1)
    # print(mode, speed)
    command = make_command(mode, speed_percent=speed, angle_degrees=degrees)
    print(f"Sending command : {command}")
    rover.send_command(command)
def captain_stop():
    rover.send_command(make_command(speed_percent=0))
command_freq = 3
def start_captain(is_captain_running):
    while is_captain_running():
        captain_act()
        # print("running captain")
        time.sleep(1/command_freq)
    # print("stopping captain")
    captain_stop()
    

In [4]:
def update_button_disables(pathfinding):
    start_pathfinder.disabled = pathfinding
    run_rover.disabled = not pathfinding
    stop_pathfinder.disabled = not pathfinding

def on_start_pathfinding(b):
    importlib.reload(pathfinder)
    pathfinder.start_pathfinder_service()
    # show_visual()
    update_button_disables(True)
start_pathfinder.on_click(on_start_pathfinding)
def on_stop_pathfinding(b):
    pathfinder.stop_pathfinder_service()
    update_button_disables(False)
stop_pathfinder.on_click(on_stop_pathfinding)

# def on_visual_toggle(change):
#     show = change['new']
#     print(show)
#     if show:
#         show_visual()
#     else:
#         plt.close('all')
# visual_button.observe(on_visual_toggle, 'value')
def on_visual(b):
    show_visual()
visual_button.on_click(on_visual)

captain_service = Service(start_captain, "Captain Service")
def rover_order_changed(change):
    global captain_service
    run = change['new']
    if run:
        captain_service.start_service()
        run_rover.button_style='danger'
        run_rover.description='Stop Rover'
    else:
        captain_service.stop_service()
        captain_service = Service(start_captain, "Captain Service")
        run_rover.button_style='success'
        run_rover.description='Run Rover'
run_rover.observe(rover_order_changed, 'value')

clear_output.on_click(lambda i: text_output.clear_output())
history = []
def on_poll_rover(b):
    def background_task():
        stat = rover.get_status()
        with text_output:
            if stat is None:
                print("No response recieved")
            else:
                print()
                print(f"at: {datetime.now().strftime('%H:%M:%S')}")
                print(json.dumps({k:stat[k] for k in stat if re.match("^current.*", k)},indent=2))
    t = threading.Thread(target=background_task)
    print("Polling Rover...")
    t.start()
poll_rover.on_click(on_poll_rover)

In [5]:
show_visual()

In [6]:
gui

In [7]:
# rover.send_command(make_command(Modes.SPIN,speed_percent=0))